The NHS dm+d contains a flag which marks if something is unavalable. A related [notebook](https://gist.github.com/sebbacon/317938750ea79e6291ae2cac51a0d31b) uncovered reimbursement against a prouct which dm+d reports as unavailable for the last five years. Here we invesitigate reimbursement of unavailable products.

In [1]:
##importing libraries
import pandas as pd
import numpy as np
from ebmdatalab import bq, maps, charts
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.dates as mdates

In [2]:
##ensure £ and pence are set right
pd.set_option('display.float_format', lambda x: '%.2f' % x)

There is a flag at vmp level relating to availability where all AMP(P) should be unavailable according to the documentation of dm+d

In [3]:
sql = '''
SELECT
  Date,
  TRIM(Principal_Supplier) AS supplier,
  bnf_name,
  presc.bnf_code,
  vmp.id,
  non_availdt,
  Items,
  actual_Cost
FROM
  ebmdatalab.hscic.prescribing_2019_04 AS presc ##latest month
Inner JOIN
  ebmdatalab.dmd.vmp AS vmp
ON
  presc.bnf_code = vmp.bnf_code
JOIN
  ebmdatalab.alex.vendors AS software #this is where the up to date vendors table is held
ON
  software.ODS = presc.practice
  AND Date = presc.month
WHERE
  non_avail = 1 ###1 = Actual Products not Available
  AND presc.bnf_code != "190203000BBAAA0" ## exclude hompathay that has only one bnf code for all VMPs
GROUP BY
  Date,
  supplier,
  bnf_name,
  presc.bnf_code,
  vmp.id,
  non_avail,
  non_availdt,
  Items,
  actual_Cost
'''



df_unavailable_vmp = bq.cached_read(sql, csv_path='amp_dmd_unavailable_vmp.csv')
df_unavailable_vmp.head(10)

,Date,supplier,bnf_name,bnf_code,id,non_availdt,Items,actual_Cost
0,2019-04-01,TPP,Alum Hydrox_Cap 475mg,0101010C0AAAAAA,316937008,2019-05-15,2,24.75
1,2019-04-01,TPP,K-Y Lubri Jelly,21140000005,34159511000001108,2019-02-01,1,2.18
2,2019-04-01,TPP,Nutramigen 2 LIPIL_Pdr,090401000BBLPA0,5174311000001105,2017-06-06,1,51.07
3,2019-04-01,TPP,SMA_Gold Prem 2 Milk,090900000BBKPA0,20594911000001102,2017-12-11,2,15.26
4,2019-04-01,TPP,Nifedipine_Cap 10mg,0206020R0AAABAB,319223009,2019-03-18,6,38.21
5,2019-04-01,Vision,Nifedipine_Cap 10mg,0206020R0AAABAB,319223009,2019-03-18,2,15.34
6,2019-04-01,EMIS,SMA_PRO Anti-reflux Pdr,090900000BBLHA0,35308011000001100,2019-03-12,2,28.27
7,2019-04-01,EMIS,Nifedipine_Cap 10mg,0206020R0AAABAB,319223009,2019-03-18,3,32.60
8,2019-04-01,TPP,Salbutamol_Inha 100mcg (200 D),0301011R0AAAAAA,35936511000001108,2009-02-25,6,12.62
9,2019-04-01,Vision,SMA_PRO Gold Prem 2 Liq,090900000BBLEA0,33665811000001102,2019-03-12,1,6.11


## April Totals

In [4]:

df_unavailable_vmp.sum()[['Items','actual_Cost']]

Items             12094
actual_Cost   240786.22
dtype: object

Aproxmately £250K was spent on items that are not available in April 2019. £113k on products (see below) that have been not available for more than 12 months.

In [5]:
###lets look at individual products to get a sense check
df_unavailable_vmp.bnf_name.unique()

array(['Alum Hydrox_Cap 475mg', 'K-Y Lubri Jelly',
       'Nutramigen 2 LIPIL_Pdr', 'SMA_Gold Prem 2 Milk',
       'Nifedipine_Cap 10mg', 'SMA_PRO Anti-reflux Pdr',
       'Salbutamol_Inha 100mcg (200 D)', 'SMA_PRO Gold Prem 2 Liq',
       'Nifedipine_Cap 5mg', 'Diltiazem HCl_Cap 60mg M/R',
       'SMA_PRO High Energy Liq', 'SMA_PRO Gold Prem 2 Pdr',
       'Acetic Acid_Ear Spy 2% 5ml',
       'Midazolam_OromucSoln 7.5mg/1.5mlPfos S/F',
       'U100 Syrg Sle Use 1ml + 12mm Needle-Ster Hypod Syrg',
       'Diazepam_Oral Soln 2mg/5ml',
       'Surg Suture W319 Ethilon Non-Absorb Ster Polyam 6 Monof',
       'Colecal & Calc_Tab 800u/1.25g',
       'Midazolam_Oromuc Soln 5mg/1ml Pfos S/F', 'Olive Oil_Ear Dps',
       'Fresubin 2kcal_Drink S/Pack', 'LittleVit_Multivit Dps',
       'Surg Suture W320 Ethilon Non-Absorb Ster Polyam 6 Monof',
       'Clonazepam_Liq Spec 2mg/5ml', 'Nutramigen 1 LIPIL_Pdr',
       'Mexiletine HCl_Tab 200mg', 'Verapamil HCl_Cap 120mg M/R',
       'Nedocromil Sod_E

In [6]:
##count the unique preparations
df_unavailable_vmp.bnf_name.nunique()

201

In [7]:
##this is a list of products that look like they may be available for checking
interesting = ['Nifedipine_Cap 10mg', 'Estradiol/Drospirenone_Tab 1mg/2mg', 'Deferiprone_Tab 500mg', 'Nifedipine_Cap 5mg', 'Olive Oil_Ear Dps', 'Colistimeth Sod_Neb Pdr 2mega u']

In [8]:
df_unavailable_vmp[df_unavailable_vmp.bnf_name.isin(interesting)].head()

,Date,supplier,bnf_name,bnf_code,id,non_availdt,Items,actual_Cost
4,2019-04-01,TPP,Nifedipine_Cap 10mg,0206020R0AAABAB,319223009,2019-03-18,6,38.21
5,2019-04-01,Vision,Nifedipine_Cap 10mg,0206020R0AAABAB,319223009,2019-03-18,2,15.34
7,2019-04-01,EMIS,Nifedipine_Cap 10mg,0206020R0AAABAB,319223009,2019-03-18,3,32.60
11,2019-04-01,EMIS,Nifedipine_Cap 5mg,0206020R0AAAAAA,319222004,2019-02-18,6,8.14
15,2019-04-01,EMIS,Nifedipine_Cap 10mg,0206020R0AAABAB,319223009,2019-03-18,2,11.13


Investigations cross referencing dm+browser, BNF and emc show that all of these idientified products are truly unavailable with this name and form. Some products have been discontinued e.g. nifedipine while others like Colistimeth Sod are possibly just cosmetic name or technical license changes

It is unfair say that all products should not be reimbursed as there may be good reason for example a pharmacy may have excess stock on the shelf and have a supply lasting 2/3 months after discontinuation.Let us look at some cut off rationale

- 2 months: System suppliers must update dm+d every 2months (although the file is updated weekly by BSA) based on eps dmd requirements
- 6 months: This is the legal validity of a prescription
- 2-12 to make an EHR curve

In [9]:
df_2months = df_unavailable_vmp.loc[(df_unavailable_vmp["non_availdt"]<="2019-04-01")]
df_2months.sum(numeric_only=True)

id            47239777004853075968.00
Items                        11263.00
actual_Cost                 227372.26
dtype: float64

In [10]:
df_6months = df_unavailable_vmp.loc[(df_unavailable_vmp["non_availdt"]<="2018-12-01")]
df_6months.sum(numeric_only=True)

id            17008990318162894848.00
Items                         2562.00
actual_Cost                 119097.21
dtype: float64

In [11]:
df_6months.groupby("supplier").sum().reset_index()

,supplier,id,Items,actual_Cost
0,EMIS,6055040237975456768,1152,53551.50
1,Microtest,571485588695112320,56,3304.08
2,TPP,9143868259653729280,1210,54702.53
3,Vision,1238596231838932736,144,7539.09


In [12]:
df_3months = df_unavailable_vmp.loc[(df_unavailable_vmp["non_availdt"]<="2019-03-01")]
df_3months.sum(numeric_only=True)

id            30483726360704397312.00
Items                         6808.00
actual_Cost                 143543.79
dtype: float64

In [13]:
df_4months = df_unavailable_vmp.loc[(df_unavailable_vmp["non_availdt"]<="2019-02-01")]
df_4months.sum(numeric_only=True)

id            27339798123941167104.00
Items                         4863.00
actual_Cost                 131229.59
dtype: float64

In [14]:
df_5months = df_unavailable_vmp.loc[(df_unavailable_vmp["non_availdt"]<="2019-01-01")]
df_5months.sum(numeric_only=True)

id            18002115257574299648.00
Items                         2666.00
actual_Cost                 121792.72
dtype: float64

In [15]:
df_7months = df_unavailable_vmp.loc[(df_unavailable_vmp["non_availdt"]<="2018-11-01")]
df_7months.sum(numeric_only=True)

id            16951857774162890752.00
Items                         2557.00
actual_Cost                 119056.97
dtype: float64

In [16]:
df_8months = df_unavailable_vmp.loc[(df_unavailable_vmp["non_availdt"]<="2018-10-01")]
df_8months.sum(numeric_only=True)

id            14837782899908151296.00
Items                         2342.00
actual_Cost                 115930.47
dtype: float64

In [17]:
df_9months = df_unavailable_vmp.loc[(df_unavailable_vmp["non_availdt"]<="2018-09-01")]
df_9months.sum(numeric_only=True)

id            14837782885387677696.00
Items                         2286.00
actual_Cost                 115367.13
dtype: float64

In [18]:
df_10months = df_unavailable_vmp.loc[(df_unavailable_vmp["non_availdt"]<="2018-08-01")]
df_10months.sum(numeric_only=True)

id            14569604140405444608.00
Items                         2158.00
actual_Cost                 113967.84
dtype: float64

In [19]:
df_11months = df_unavailable_vmp.loc[(df_unavailable_vmp["non_availdt"]<="2018-07-01")]
df_11months.sum(numeric_only=True)

id            14382797163449012224.00
Items                         2137.00
actual_Cost                 113643.99
dtype: float64

In [20]:
df_12months = df_unavailable_vmp.loc[(df_unavailable_vmp["non_availdt"]<="2018-06-01")]
df_12months.sum(numeric_only=True)

id            14345662728533942272.00
Items                         2121.00
actual_Cost                 113526.72
dtype: float64

£113k was spent in April 2019 on products that have been discontinued greater than 12 months ago. 

In [21]:
df_12months.groupby("bnf_name").sum().sort_values(by = 'Items',ascending=False).head(10)

,id,Items,actual_Cost
bnf_name,,,
Midazolam_Oromuc Soln 10mg/2ml Pfos S/F,1770164344000113152,254,30423.39
Nedocromil Sod_Eye Dps Aq 2%,24128617511,226,1025.66
U100 Syrg Sle Use 0.5ml + 12mm Needle-Ster Hypod Syrg,386447721000121408,179,2191.58
U100 Syrg Sle Use 1ml + 12mm Needle-Ster Hypod Syrg,320335812000102208,158,2136.46
Verapamil HCl_Cap 120mg M/R,1608860484000048128,126,803.44
Verapamil HCl_Cap 240mg M/R,10184000288,88,771.23
Midazolam_Oromuc Soln 5mg/1ml Pfos S/F,799987517000052096,86,11187.30
Midazolam_OromucSoln 7.5mg/1.5mlPfos S/F,748928884000048640,75,9594.23
Diazepam_Oral Soln 2mg/5ml,16381149306,73,8534.44


Looking at some examples of stuff that is unavailble more than 12 months ago
- Erythromycin_Cap E/C 250mg this does not appear to be available but ec tabs are.
- Verapamil HCl_Cap 120mg M/R caps do not appear to be available but tabs are. Univer previously made by Teva was avaiable but documented [reports](http://mm.wirral.nhs.uk/document_uploads/newsletters/TABLETSDec2015Issue21WirralFINAL.pdf) of it not being available.


In [22]:
df_12months.groupby("supplier").sum().reset_index()

,supplier,id,Items,actual_Cost
0,EMIS,5665011634717020160,974,51763.89
1,Microtest,551138877377710208,53,3143.69
2,TPP,7135073220487595008,990,51696.23
3,Vision,994438995951802496,104,6922.92


The split appears even between suppliers in terms of absolute cost. Let us see if there is a difference in product mix between EMIS and SystmOne

a) by cost

In [23]:
df_emis = df_unavailable_vmp[df_unavailable_vmp["supplier"].str.contains("emis", case=False)]
df_emis.groupby("bnf_name").sum().sort_values(by = 'actual_Cost',ascending=False).head(10)


,id,Items,actual_Cost
bnf_name,,,
SMA_PRO High Energy Liq,3575281944000110080,198,21338.14
Midazolam_Oromuc Soln 10mg/2ml Pfos S/F,799978117000052096,125,16986.77
Midazolam_Oromuc Soln 5mg/1ml Pfos S/F,425525275000027328,56,7710.58
Nifedipine_Cap 10mg,114601060231,1323,5986.58
Midazolam_OromucSoln 2.5mg/0.5mlPfos S/F,255313665000016672,23,5742.78
Midazolam_OromucSoln 7.5mg/1.5mlPfos S/F,340422220000021888,33,4114.93
Nifedipine_Cap 5mg,96085823204,892,3481.12
Diazepam_Liq Spec 10mg/5ml,131890665000016576,18,2913.41
K-Y Lubri Jelly,4133300831000127488,360,2544.65


In [24]:
df_tpp = df_unavailable_vmp[df_unavailable_vmp["supplier"].str.contains("tpp", case=False)]
df_tpp.groupby("bnf_name").sum().sort_values(by = 'actual_Cost',ascending=False).head(10)

,id,Items,actual_Cost
bnf_name,,,
SMA_PRO High Energy Liq,3334637967000102912,195,20948.31
Midazolam_Oromuc Soln 10mg/2ml Pfos S/F,714874062000046336,97,11118.67
SMA_PRO Gold Prem 2 Pdr,3534920655000111104,408,10437.18
Nifedipine_Cap 10mg,133115994753,1693,8094.14
Diazepam_Oral Soln 2mg/5ml,15096353282,69,7698.67
Mexiletine HCl_Tab 200mg,126603288000008832,9,5540.09
Midazolam_OromucSoln 7.5mg/1.5mlPfos S/F,340422220000021888,37,5107.03
SMA_PRO Gold Prem 2 Liq,1009974330000033536,72,4747.22
Clobazam_Liq Spec 5mg/5ml,125529165000016544,18,4185.69


b) by items

In [25]:
df_emis.groupby("bnf_name").sum().sort_values(by = 'Items',ascending=False).head(10)

,id,Items,actual_Cost
bnf_name,,,
Nifedipine_Cap 10mg,114601060231,1323,5986.58
Nifedipine_Cap 5mg,96085823204,892,3481.12
K-Y Lubri Jelly,4133300831000127488,360,2544.65
Surg Suture W320 Ethilon Non-Absorb Ster Polyam 6 Monof,306264462000046720,305,500.47
Surg Suture W319 Ethilon Non-Absorb Ster Polyam 6 Monof,298964251000045568,303,468.61
Acetic Acid_Ear Spy 2% 5ml,19523100236,297,1595.62
SMA_PRO High Energy Liq,3575281944000110080,198,21338.14
Nedocromil Sod_Eye Dps Aq 2%,12890631273,137,647.33
Midazolam_Oromuc Soln 10mg/2ml Pfos S/F,799978117000052096,125,16986.77


In [26]:
df_tpp.groupby("bnf_name").sum().sort_values(by = 'Items',ascending=False).head(10)

,id,Items,actual_Cost
bnf_name,,,
Nifedipine_Cap 10mg,133115994753,1693,8094.14
Nifedipine_Cap 5mg,73421060920,669,2806.10
SMA_PRO Gold Prem 2 Pdr,3534920655000111104,408,10437.18
Surg Suture W319 Ethilon Non-Absorb Ster Polyam 6 Monof,328131495000049920,385,1006.21
Surg Suture W320 Ethilon Non-Absorb Ster Polyam 6 Monof,320848484000048896,297,660.63
Acetic Acid_Ear Spy 2% 5ml,17537700212,271,1233.30
K-Y Lubri Jelly,3108515501000096768,228,1636.42
SMA_PRO High Energy Liq,3334637967000102912,195,20948.31
Midazolam_Oromuc Soln 10mg/2ml Pfos S/F,714874062000046336,97,11118.67


## Other ideas
look at where there is a unique bnf code for AMP e.g. metabet and salbutamol